In [39]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from collections import Counter
import matplotlib.pylab as plt

In [3]:
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""

In [4]:
# Import des questions

In [117]:
answers_grand_debat_LTE_raw = pd.read_csv('raw/large_datasets_ELF/LA_TRANSITION_ECOLOGIQUE.csv', encoding="utf-8").drop_duplicates(keep=False)
answers_grand_debat_FDP_raw = pd.read_csv('raw/large_datasets_ELF/LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES.csv', encoding="utf-8").drop_duplicates(keep=False)
answers_grand_debat_ORG_raw = pd.read_csv('raw/large_datasets_ELF/ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS.csv', encoding="utf-8").drop_duplicates(keep=False)
answers_grand_debat_DEC_raw = pd.read_csv('raw/large_datasets_ELF/DEMOCRATIE_ET_CITOYENNETE.csv', encoding="utf-8").drop_duplicates(keep=False)

In [173]:
def keep_popular_hashtags_by_question(dataframe):
    allowed_categories_gda = []
    print(dataframe['id_gda'].unique())
    for question in dataframe['id_gda'].unique():
        allowed_categories_tuples = Counter(dataframe[dataframe['id_gda'] == question]['tag_elf']).most_common(20)
        allowed_categories_gda += [(question, element[0], element[1]) for element in allowed_categories_tuples]
    for question in dataframe['id_elf'].unique():    
        allowed_categories_tuples = Counter(dataframe[dataframe['id_elf'] == question]['tag_elf']).most_common(20)
        allowed_categories = [element[0] for element in allowed_categories_tuples]
        dataframe = dataframe[(dataframe['id_elf'] != 'question') | (dataframe['tag_elf'].isin(allowed_categories))]
    return dataframe, allowed_categories_gda

def process_LGD_dataset(raw_dataframe):
    """
    """
    answers_annotees = pd.read_csv('raw/actions.csv', encoding="utf-8").rename(columns={'Contribution': 'reference'})
    print(raw_dataframe.shape)
    raw_dataframe = pd.merge(answers_annotees, raw_dataframe, on='reference')
    if raw_dataframe.shape[0] == 0:
        print("Empty records when matching on annotations")
        return None, None
    
    questions_dictionary = pd.read_csv('raw/match_questions_ELF_LGA.csv', header=0)
    raw_dataframe = pd.merge(raw_dataframe, questions_dictionary, left_on='Question', right_on='id_gda')
    print(raw_dataframe.shape)

    gda_dictionary = pd.read_csv('raw/matching_gda_id1_id2.csv', header=0)
    raw_dataframe = pd.merge(raw_dataframe, gda_dictionary, left_on='id_gda', right_on='id1_gda')
    print(raw_dataframe.shape)
    
    raw_dataframe["corpus"] = None
    
    for index, row in raw_dataframe.iterrows():
        raw_dataframe.at[index,'corpus'] = raw_dataframe.loc[index][raw_dataframe.filter(like=row['id2_gda']).columns[0]]

    raw_dataframe = raw_dataframe[['reference', 'id_elf', 'corpus', 'Categorie', 'Annotateur', 'id_gda']]
    
    tags_dictionary = pd.read_csv('raw/matching_tags_lga_elf.csv', sep=';')
    raw_dataframe = pd.merge(raw_dataframe, tags_dictionary, left_on=['id_gda', 'Categorie'], right_on=['id_lga', 'tag_lga'])
    print(raw_dataframe.shape)
    
    raw_dataframe, allowed_categories_gda = keep_popular_hashtags_by_question(raw_dataframe)
    print(raw_dataframe.shape)
    
    raw_dataframe = pd.DataFrame({'count' : raw_dataframe.groupby( ['reference', 'tag_elf', 'corpus', 'id_elf'] ).apply(lambda x: x.Annotateur.nunique())}).reset_index()
    print(raw_dataframe.shape)
    
    return raw_dataframe.rename(columns={'reference': 'contribution', 'corpus': 'answer', 'tag_elf': 'target', 'id_elf' : 'question'}), allowed_categories_gda
    

In [174]:
answers_grand_debat_GDA_DEC, allowed_categories_gda_DEC = process_LGD_dataset(answers_grand_debat_DEC_raw)
answers_grand_debat_GDA_FDP, allowed_categories_gda_FDP = process_LGD_dataset(answers_grand_debat_FDP_raw)
# answers_grand_debat_GDA_ORG, allowed_categories_gda_ORG = process_LGD_dataset(answers_grand_debat_ORG_raw)
answers_grand_debat_GDA_LTE, allowed_categories_gda_LTE = process_LGD_dataset(answers_grand_debat_LTE_raw)

(73746, 48)
(5915, 55)
(5915, 57)
(2878, 9)
[127]
(2878, 9)
(926, 5)
(126165, 19)
(23033, 26)
(23033, 28)
(17454, 9)
[162 163]
(17454, 9)
(6804, 5)
(99343, 27)
(20296, 34)
(20296, 36)
(17896, 9)
[149]
(17896, 9)
(5334, 5)


In [175]:
answers_grand_debat_GDA = pd.concat([answers_grand_debat_GDA_FDP, answers_grand_debat_GDA_DEC, answers_grand_debat_GDA_LTE])
answers_grand_debat_GDA['source'] = 'GDA'

In [176]:
answers_grand_debat_ELF = pd.read_csv("preprocessed/elf_tags_sample.csv", sep=";").rename(columns={'reference': 'contribution', 'corpus': 'answer', 'tag': 'target', 'id_elf' : 'question'})
answers_grand_debat_ELF['source'] = 'ELF'

In [177]:
answers_grand_debat_ELF.head()

,contribution,target,answer,question,count,source
0,37 240,Le peuple pas responsable du problème écologique,Stopper toutes ces taxes et donner une autre i...,TE13,1,ELF
1,12 970,Etre plus à l'écoute,Avoir des représentants actif qui remontent et...,OE11,1,ELF
2,12 970,Plus de transparence,Avoir des représentants actif qui remontent et...,OE11,1,ELF
3,13 210,Incitations financières au lieu de taxes,Il faudrait pousser la création de beaucoup pl...,TE13,1,ELF
4,13 210,Développer les énergies renouvelables,Il faudrait pousser la création de beaucoup pl...,TE13,1,ELF


In [178]:
answers_grand_debat = pd.concat([answers_grand_debat_ELF, answers_grand_debat_GDA])
Counter(answers_grand_debat['target']).most_common(20)

[('Tri sélectif, compost, réduction déchets', 1326),
 ('Transports plus verts ou réduits', 990),
 ('Plus de transparence', 881),
 ('Consommation réduite ou vertueuse (bio, circuit court, local)', 743),
 ('Impôt symbolique pour tous', 730),
 ('Réduction consommation énergie, eau', 704),
 ('Plus de tranches IR progressives', 593),
 ('Régime alimentaire', 533),
 ('Plus de pédagogie', 437),
 ("Plus d'information sur la dépense des impôts", 421),
 ('Revoir niches', 384),
 ('Meilleure isolation, changement chauffage/chaudière', 373),
 ('Lutte évasion, fraude, optimisation', 346),
 ('Application loi 1905 / Séparation stricte Etat-religions', 319),
 ('Rétablir ISF', 316),
 ('Cultiver/Jardiner bio, sans pesticide', 257),
 ('Taxer les riches', 251),
 ('Utilisation énergies Renouvelables', 234),
 ('Simplifier', 223),
 ('Taxer les multinationales, GAFAM', 218)]

In [179]:
answers_grand_debat_sample = answers_grand_debat.groupby( ['question', 'answer'] ).apply(lambda x: x.target.unique()).reset_index()

In [180]:
answers_grand_debat_sample = answers_grand_debat_sample.rename(columns={0: 'target'})

In [181]:
answers_grand_debat_sample.head()

,question,answer,target
0,DC11,Il faut que les citoyens qui ne participent pa...,"[vote obligatoire, plus de référendums]"
1,DC11,Mettre en place un vote par internet. Améliore...,"[nouveau mode de scrutin (proportionnelle, etc..."
2,DC11,Modifier la manière de voter pour que les élec...,"[nouveau mode de scrutin (proportionnelle, etc.)]"
3,DC11,Rendre les politiques plus crédibles et avoir ...,[respect des engagements des dirigeants politi...
4,DC13,- Contrôler les mosquées et leur fonctionnemen...,[Encadrement des religions (écoles culturelles...


In [162]:
answers_grand_debat_sample.to_json('preprocessed/reponses_annotees_multilabel.json', orient = "records")